<h1>Оглавление<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Data-loader" data-toc-modified-id="Data-loader-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Data loader</a></span></li></ul></div>

# Подключение библиотек

In [1]:
import fiftyone as fo
import fiftyone.zoo as foz

# Download and load the validation split of COCO-2017
fo_dataset = foz.load_zoo_dataset("coco-2017", split="validation",
                               dataset_dir='/mnt/heap')

Found annotations at '/mnt/heap/raw/instances_val2017.json'
Images already downloaded
Existing download of split 'validation' is sufficient
Loading 'coco-2017' split 'validation'
 100% |███████████████| 5000/5000 [22.3s elapsed, 0s remaining, 222.4 samples/s]      
Dataset 'coco-2017-validation' created


session = fo.launch_app(dataset)

In [2]:
fo_dataset.compute_metadata()

## Data set

In [3]:
import torch
import fiftyone.utils.coco as fouc
from PIL import Image


class FiftyOneTorchDataset(torch.utils.data.Dataset):
    """A class to construct a PyTorch dataset from a FiftyOne dataset.
    
    Args:
        fiftyone_dataset: a FiftyOne dataset or view that will be used for training or testing
        transforms (None): a list of PyTorch transforms to apply to images and targets when loading
        gt_field ("ground_truth"): the name of the field in fiftyone_dataset that contains the 
            desired labels to load
        classes (None): a list of class strings that are used to define the mapping between
            class names and indices. If None, it will use all classes present in the given fiftyone_dataset.
    """

    def __init__(
        self,
        fiftyone_dataset,
        transforms=None,
        gt_field="ground_truth",
        classes=None,
    ):
        self.samples = fiftyone_dataset
        self.transforms = transforms
        self.gt_field = gt_field

        self.img_paths = self.samples.values("filepath")

        self.classes = classes
        if not self.classes:
            # Get list of distinct labels that exist in the view
            self.classes = self.samples.distinct(
                "%s.detections.label" % gt_field
            )

        if self.classes[0] != "background":
            self.classes = ["background"] + self.classes

        self.labels_map_rev = {c: i for i, c in enumerate(self.classes)}

    def __getitem__(self, idx):
        img_path = self.img_paths[idx]
        sample = self.samples[img_path]
        metadata = sample.metadata
        img = Image.open(img_path).convert("RGB")

        boxes = []
        detections = sample[self.gt_field].detections
        img_has_person = 0
        for det in detections:
            category_id = self.labels_map_rev[det.label]
            coco_obj = fouc.COCOObject.from_detection(
                det, metadata, category_id=category_id,
            )
            x, y, w, h = coco_obj.bbox
            
            if det.label == 'person':
                boxes.append([x, y, x + w, y + h])
                img_has_person = 1

        target = {}
        '''
        print(boxes)
        print('==================================')
        box = sorted(boxes)[0]  
        print(box)
        '''
        box = sorted(boxes)[0]  
        target["box"] = torch.as_tensor(box, dtype=torch.float32)                              
        target["img_has_person"] = img_has_person

        if self.transforms is not None:
            img = self.transforms(img)

        return img, target

    def __len__(self):
        return len(self.img_paths)

    def get_classes(self):
        return self.classes

In [4]:
from fiftyone import ViewField as F

person_list = ["person"]
person_view = fo_dataset.filter_labels("ground_truth",
        F("label").is_in(person_list))

print(len(person_view))

2693


## Data loader

In [5]:
import torchvision.transforms as T

img_size = (256, 256)
train_transforms = T.Compose([T.Resize(img_size),
                              T.ToTensor()])
test_transforms = T.Compose([T.Resize(img_size),
                             T.ToTensor()])

In [6]:
# split the dataset in train and test set
train_view = person_view.take(len(person_view) * 0.75, seed=51)
test_view = person_view.exclude([s.id for s in train_view])

In [7]:
torch_dataset = FiftyOneTorchDataset(train_view, train_transforms,
        classes=person_list)
torch_dataset_test = FiftyOneTorchDataset(test_view, test_transforms, 
        classes=person_list)

In [8]:
len(torch_dataset)

2019

In [9]:
len(torch_dataset_test)

674

In [19]:
# define training and validation data loaders
data_loader = torch.utils.data.DataLoader(
        torch_dataset, batch_size=100)
    
data_loader_test = torch.utils.data.DataLoader(
        torch_dataset_test, batch_size=1, shuffle=False)

# Train loop

In [11]:
from itertools import product
from pytorch_helper import PyTorchHelper

In [12]:
loss_function_xy = torch.nn.MSELoss()
loss_function_bce = torch.nn.CrossEntropyLoss()

In [22]:
from pytorch_helper import PyTorchHelper

def find_hyperparameters(config):
    learning_rates = [1e-1]
    anneal_coeff = 0.2
    anneal_epochs = [5]
    regs = config['regs']
    optimizers = config['optimizers']

    batch_size = 64
    epoch_num = 5

    run_record = {} 
    
    helper = PyTorchHelper(8,  None)

    for lr, reg, anneal_epoch, optimizer in product(learning_rates, regs, anneal_epochs, optimizers):
        lenet_model = config['model']() # base_lenet()

        loss = None

        optimizer = optimizer(lenet_model.parameters(), lr=lr, weight_decay=reg)
        scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=anneal_epoch, gamma=anneal_coeff)

        loss_history = helper.train_model(config['model_name'],lenet_model, data_loader, data_loader_test, loss, optimizer, epoch_num, scheduler)

    return loss_history

In [14]:
import torch
import torch.nn as nn
import torch.optim as optim

from torchvision.models import resnet18

In [15]:
def net():
    model = resnet18(pretrained=True)
    model.fc = nn.Linear(in_features=512, out_features=5)
    return model

In [16]:
config = {
    'regs': [0.0001],
    'optimizers': [optim.SGD],
    'model': net,
    'model_name': 'best_lenet'    
}

In [ ]:
loss_history = find_hyperparameters(config)

Epoch 0/4
----------
Step 0/21 Loss 233.6032257080078
Step 1/21 Loss 217.22764587402344
Step 2/21 Loss 204.3895721435547
Step 3/21 Loss 199.2304229736328
Step 4/21 Loss 190.92538452148438
Step 5/21 Loss 182.36219787597656
Step 6/21 Loss 185.94503784179688
Step 7/21 Loss 182.31068420410156
Step 8/21 Loss 165.11727905273438
Step 9/21 Loss 167.14474487304688
Step 10/21 Loss 174.6124267578125
Step 11/21 Loss 153.5272216796875
Step 12/21 Loss 160.06289672851562
Step 13/21 Loss 144.33636474609375
Step 14/21 Loss 140.5021514892578
Step 15/21 Loss 131.0885772705078
Step 16/21 Loss 129.2145538330078
Step 17/21 Loss 133.4313201904297
Step 18/21 Loss 127.82332611083984
Step 19/21 Loss 123.5042953491211
Step 20/21 Loss 116.2641372680664
Average loss: 173.131210
Epoch 1/4
----------
Step 0/21 Loss 124.89096069335938
Step 1/21 Loss 117.78799438476562
Step 2/21 Loss 119.39697265625
Step 3/21 Loss 114.70049285888672
Step 4/21 Loss 115.88742065429688
Step 5/21 Loss 111.16327667236328
Step 6/21 Loss 123

In [18]:
ten1 = 

SyntaxError: invalid syntax (<ipython-input-18-10225f8106b5>, line 1)